<a href="https://colab.research.google.com/github/adriansletten/CV_test/blob/main/cv_test_yolov8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CV test
This file contains the code used for finetuning a yolo-v8 model on the lions and hippos dataset.


## Step 1: Install required packages


In [ ]:
!pip install ultralytics==8.0.196
!pip install roboflow

## Step 2: Load dataset from roboflow

In [ ]:
from google.colab import userdata
api_key = userdata.get('roboflow_api_key')

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key=api_key)
project = rf.workspace("adriansletten").project("lions_and_hippos")
dataset = project.version(2).download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Exporting format yolov8 in progress : 85.0%
Version export complete for yolov8 format



Extracting Dataset Version Zip to lions_and_hippos-2 in yolov8:: 100%|██████████| 1380/1380 [00:00<00:00, 1719.01it/s]


## Step 3: Train model


In [ ]:
!yolo task=detect \
mode=train \
model=yolov8s.pt \
data={dataset.location}/data.yaml \
epochs=100 \
imgsz=640


New https://pypi.org/project/ultralytics/8.0.220 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/lions_and_hippos-2/data.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augme

## Step 4: Evaluate model

In [ ]:
!yolo task=detect \
mode=val \
model=/content/runs/detect/train/weights/best.pt \
data={dataset.location}/data.yaml


Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs
val: Scanning /content/lions_and_hippos-2/valid/labels.cache... 55 images, 1 backgrounds, 0 corrupt: 100% 55/55 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 4/4 [00:03<00:00,  1.13it/s]
                   all         55         54      0.993      0.964      0.991      0.848
          hippopotamus         55         40      0.993          1      0.995      0.964
                  lion         55         14      0.992      0.929      0.986      0.732
Speed: 4.4ms preprocess, 19.0ms inference, 0.0ms loss, 4.2ms postprocess per image
Results saved to runs/detect/val9
💡 Learn more at https://docs.ultralytics.com/modes/val


## Step 5: Test model

In [ ]:
!yolo task=detect \
mode=predict \
model=/content/runs/detect/train/weights/best.pt \
conf=0.25 \
source={dataset.location}/test/images

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs

image 1/29 /content/lions_and_hippos-2/test/images/hippopotamus-102_jpg.rf.ead18b78e716c5b216f29ee88231df48.jpg: 416x640 1 hippopotamus, 117.1ms
image 2/29 /content/lions_and_hippos-2/test/images/hippopotamus-103_jpg.rf.2a9472670c0a28c93dd7d6d0e0d4dfea.jpg: 640x640 1 hippopotamus, 17.2ms
image 3/29 /content/lions_and_hippos-2/test/images/hippopotamus-110_jpg.rf.d051ca2b8b0610206634a719e814c3c8.jpg: 448x640 1 hippopotamus, 70.1ms
image 4/29 /content/lions_and_hippos-2/test/images/hippopotamus-115_jpg.rf.cbe1155b9fb4135393ad9a004355f7d5.jpg: 480x640 1 hippopotamus, 74.3ms
image 5/29 /content/lions_and_hippos-2/test/images/hippopotamus-124_jpg.rf.b652e4622f4140fe079eca7f9e25b4f1.jpg: 640x448 1 hippopotamus, 66.2ms
image 6/29 /content/lions_and_hippos-2/test/images/hippopotamus-128_jpg.rf.39c576759adbead2fe22bcde30fca56c.j

## Step 6: Store results

In [ ]:
!zip -r /content/runs.zip /content/runs

## Step 7: Deploy model to Roboflow

In [ ]:
project.version(dataset.version).deploy(model_type="yolov8", model_path=f"/content/runs/detect/train/", )

View the status of your deployment at: https://app.roboflow.com/adriansletten/lions_and_hippos/2
Share your model with the world at: https://universe.roboflow.com/adriansletten/lions_and_hippos/model/2
